In [2]:
from collections import defaultdict
import json
import os
import pickle
import statistics

from requests import Session

import matplotlib.pyplot as plt
import pandas as pd

import statbotics

%matplotlib notebook

In [3]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

sb = statbotics.Statbotics()

In [4]:
def dump(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path, "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load(file):
    with open(file, "rb") as f:
        return pickle.load(f)


def dump_cache(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path + "/data.p", "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load_cache(file):
    with open(file + "/data.p", "rb") as f:
        return pickle.load(f)

    
    
def get_tba(url):
    if os.path.exists("../../backend/cache/" + url + "/data.p"):
        # Cache Hit
        return load_cache("../../backend/cache/" + url)

    response = session.get(read_prefix + url)
    data = response.json()

    # Cache Miss
    dump_cache("../../backend/cache/" + url, data)
    return data

In [5]:
events = get_tba("events/2023")
events = [(e["key"], e["week"]) for e in events]

print(len(events))

all_matches = []
for event, week in events:
    all_matches.extend([x, week] for x in get_tba("event/" + event + "/matches"))
    
print(len(all_matches))

179
10449


In [6]:
year = sb.get_year(2023)
teams = sb.get_team_years(year=2023, limit=10000)

TOTAL_MEAN, TOTAL_SD = year["score_mean"], year["score_sd"]
starting_epas = {t["team"]: (t["epa_start"] - TOTAL_MEAN / 3) / TOTAL_SD for t in teams}
locations = {t["team"]: (t["country"], t["state"], t["district"]) for t in teams}
print(starting_epas[254], starting_epas[5511])

0.9612851952770209 0.03689827429609454


In [106]:
def process_match(m, w):
    rb = m["score_breakdown"]["red"]
    bb = m["score_breakdown"]["blue"]
    
    out = {
        "week": w + 1, # TBA off by one
        "event": m["event_key"],
        "key": m["key"],
        "time": m["actual_time"],
        "playoff": m["comp_level"] != "qm",
        "red": [int(x[3:]) for x in m["alliances"]["red"]["team_keys"]],
        "blue": [int(x[3:]) for x in m["alliances"]["blue"]["team_keys"]],
        "red_auto_cycles": rb["autoGamePieceCount"],
        "blue_auto_cycles": bb["autoGamePieceCount"],
        "red_auto_points": rb["autoGamePiecePoints"],
        "blue_auto_points": bb["autoGamePiecePoints"],
        "red_teleop_cycles": rb["teleopGamePieceCount"] - rb["autoGamePieceCount"],
        "blue_teleop_cycles": bb["teleopGamePieceCount"] - bb["autoGamePieceCount"],
        "red_teleop_points": rb["teleopGamePiecePoints"],
        "blue_teleop_points": bb["teleopGamePiecePoints"],
        "red_links": rb["linkPoints"] / 5,
        "blue_links": bb["linkPoints"] / 5,
        "red_link_points": rb["linkPoints"],
        "blue_link_points": bb["linkPoints"],
        "red_total_cycles": rb["teleopGamePieceCount"],
        "blue_total_cycles": bb["teleopGamePieceCount"],
        "red_total_points": rb["autoGamePiecePoints"] + rb["teleopGamePiecePoints"] + rb["linkPoints"],
        "blue_total_points":  bb["autoGamePiecePoints"] + bb["teleopGamePiecePoints"] + bb["linkPoints"],
        "red_auto_bot_cubes": len([x for x in rb["autoCommunity"]["B"] if x == "Cube"]),
        "blue_auto_bot_cubes": len([x for x in bb["autoCommunity"]["B"] if x == "Cube"]),
        "red_auto_mid_cubes": len([x for x in rb["autoCommunity"]["M"] if x == "Cube"]),
        "blue_auto_mid_cubes": len([x for x in bb["autoCommunity"]["M"] if x == "Cube"]),
        "red_auto_top_cubes": len([x for x in rb["autoCommunity"]["T"] if x == "Cube"]),
        "blue_auto_top_cubes": len([x for x in bb["autoCommunity"]["T"] if x == "Cube"]),
        "red_auto_bot_cones": len([x for x in rb["autoCommunity"]["B"] if x == "Cone"]),
        "blue_auto_bot_cones": len([x for x in bb["autoCommunity"]["B"] if x == "Cone"]),
        "red_auto_mid_cones": len([x for x in rb["autoCommunity"]["M"] if x == "Cone"]),
        "blue_auto_mid_cones": len([x for x in bb["autoCommunity"]["M"] if x == "Cone"]),
        "red_auto_top_cones": len([x for x in rb["autoCommunity"]["T"] if x == "Cone"]),
        "blue_auto_top_cones": len([x for x in bb["autoCommunity"]["T"] if x == "Cone"]),
        "red_auto_bot_cycles": len([x for x in rb["autoCommunity"]["B"] if x != "None"]),
        "blue_auto_bot_cycles": len([x for x in bb["autoCommunity"]["B"] if x != "None"]),
        "red_auto_mid_cycles": len([x for x in rb["autoCommunity"]["M"] if x != "None"]),
        "blue_auto_mid_cycles": len([x for x in bb["autoCommunity"]["M"] if x != "None"]),
        "red_auto_top_cycles": len([x for x in rb["autoCommunity"]["T"] if x != "None"]),
        "blue_auto_top_cycles": len([x for x in bb["autoCommunity"]["T"] if x != "None"]),
    }
    
    out["red_teleop_bot_cubes"] = len([x for x in rb["teleopCommunity"]["B"] if x == "Cube"]) - out["red_auto_bot_cubes"]
    out["blue_teleop_bot_cubes"] = len([x for x in bb["teleopCommunity"]["B"] if x == "Cube"]) - out["blue_auto_bot_cubes"]
    out["red_teleop_mid_cubes"] = len([x for x in rb["teleopCommunity"]["M"] if x == "Cube"]) - out["red_auto_mid_cubes"]
    out["blue_teleop_mid_cubes"] = len([x for x in bb["teleopCommunity"]["M"] if x == "Cube"]) - out["blue_auto_mid_cubes"]
    out["red_teleop_top_cubes"] = len([x for x in rb["teleopCommunity"]["T"] if x == "Cube"]) - out["red_auto_top_cubes"]
    out["blue_teleop_top_cubes"] = len([x for x in bb["teleopCommunity"]["T"] if x == "Cube"]) - out["blue_auto_top_cubes"]
    out["red_teleop_bot_cones"] = len([x for x in rb["teleopCommunity"]["B"] if x == "Cone"]) - out["red_auto_bot_cones"]
    out["blue_teleop_bot_cones"] = len([x for x in bb["teleopCommunity"]["B"] if x == "Cone"]) - out["blue_auto_bot_cones"]
    out["red_teleop_mid_cones"] = len([x for x in rb["teleopCommunity"]["M"] if x == "Cone"]) - out["red_auto_mid_cones"]
    out["blue_teleop_mid_cones"] = len([x for x in bb["teleopCommunity"]["M"] if x == "Cone"]) - out["blue_auto_mid_cones"]
    out["red_teleop_top_cones"] = len([x for x in rb["teleopCommunity"]["T"] if x == "Cone"]) - out["red_auto_top_cones"]
    out["blue_teleop_top_cones"] = len([x for x in bb["teleopCommunity"]["T"] if x == "Cone"]) - out["blue_auto_top_cones"]
    out["red_teleop_bot_cycles"] = len([x for x in rb["teleopCommunity"]["B"] if x != "None"]) - out["red_auto_bot_cycles"]
    out["blue_teleop_bot_cycles"] = len([x for x in bb["teleopCommunity"]["B"] if x != "None"]) - out["blue_auto_bot_cycles"]
    out["red_teleop_mid_cycles"] = len([x for x in rb["teleopCommunity"]["M"] if x != "None"]) - out["red_auto_mid_cycles"]
    out["blue_teleop_mid_cycles"] = len([x for x in bb["teleopCommunity"]["M"] if x != "None"]) - out["blue_auto_mid_cycles"]
    out["red_teleop_top_cycles"] = len([x for x in rb["teleopCommunity"]["T"] if x != "None"]) - out["red_auto_top_cycles"]
    out["blue_teleop_top_cycles"] = len([x for x in bb["teleopCommunity"]["T"] if x != "None"]) - out["blue_auto_top_cycles"]
    
    out["red_auto_cube_cycles"] = out["red_auto_bot_cubes"] + out["red_auto_mid_cubes"] + out["red_auto_top_cubes"]
    out["blue_auto_cube_cycles"] = out["blue_auto_bot_cubes"] + out["blue_auto_mid_cubes"] + out["blue_auto_top_cubes"]
    out["red_auto_cone_cycles"] = out["red_auto_bot_cones"] + out["red_auto_mid_cones"] + out["red_auto_top_cones"]
    out["blue_auto_cone_cycles"] = out["blue_auto_bot_cones"] + out["blue_auto_mid_cones"] + out["blue_auto_top_cones"]
    out["red_auto_cube_points"] = 3 * out["red_auto_bot_cubes"] + 4 * out["red_auto_mid_cubes"] + 6 * out["red_auto_top_cubes"]
    out["blue_auto_cube_points"] = 3 * out["blue_auto_bot_cubes"] + 4 * out["blue_auto_mid_cubes"] + 6 * out["blue_auto_top_cubes"]
    out["red_auto_cone_points"] = 3 * out["red_auto_bot_cones"] + 4 * out["red_auto_mid_cones"] + 6 * out["red_auto_top_cones"]
    out["blue_auto_cone_points"] = 3 * out["blue_auto_bot_cones"] + 4 * out["blue_auto_mid_cones"] + 6 * out["blue_auto_top_cones"]
    
    out["red_teleop_cube_cycles"] = out["red_teleop_bot_cubes"] + out["red_teleop_mid_cubes"] + out["red_teleop_top_cubes"]
    out["blue_teleop_cube_cycles"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_mid_cubes"] + out["blue_teleop_top_cubes"]
    out["red_teleop_cone_cycles"] = out["red_teleop_bot_cones"] + out["red_teleop_mid_cones"] + out["red_teleop_top_cones"]
    out["blue_teleop_cone_cycles"] = out["blue_teleop_bot_cones"] + out["blue_teleop_mid_cones"] + out["blue_teleop_top_cones"]
    out["red_teleop_cube_points"] = 2 * out["red_teleop_bot_cubes"] + 3 * out["red_teleop_mid_cubes"] + 5 * out["red_teleop_top_cubes"]
    out["blue_teleop_cube_points"] = 2 * out["blue_teleop_bot_cubes"] + 3 * out["blue_teleop_mid_cubes"] + 5 * out["blue_teleop_top_cubes"]
    out["red_teleop_cone_points"] = 2 * out["red_teleop_bot_cones"] + 3 * out["red_teleop_mid_cones"] + 5 * out["red_teleop_top_cones"]
    out["blue_teleop_cone_points"] = 2 * out["blue_teleop_bot_cones"] + 3 * out["blue_teleop_mid_cones"] + 5 * out["blue_teleop_top_cones"]
    
    out["red_cube_cycles"] = out["red_auto_cube_cycles"] + out["red_teleop_cube_cycles"]
    out["blue_cube_cycles"] = out["blue_auto_cube_cycles"] + out["blue_teleop_cube_cycles"]
    out["red_cone_cycles"] = out["red_auto_cone_cycles"] + out["red_teleop_cone_cycles"]
    out["blue_cone_cycles"] = out["blue_auto_cone_cycles"] + out["blue_teleop_cone_cycles"]
    out["red_cube_points"] = out["red_auto_cube_points"] + out["red_teleop_cube_points"]
    out["blue_cube_points"] = out["blue_auto_cube_points"] + out["blue_teleop_cube_points"]
    out["red_cone_points"] = out["red_auto_cone_points"] + out["red_teleop_cone_points"]
    out["blue_cone_points"] = out["blue_auto_cone_points"] + out["blue_teleop_cone_points"]
    
    out["red_auto_bot_cycles"] = out["red_auto_bot_cubes"] + out["red_auto_bot_cones"]
    out["blue_auto_bot_cycles"] = out["blue_auto_bot_cubes"] + out["blue_auto_bot_cones"]
    out["red_auto_mid_cycles"] = out["red_auto_mid_cubes"] + out["red_auto_mid_cones"]
    out["blue_auto_mid_cycles"] = out["blue_auto_mid_cubes"] + out["blue_auto_mid_cones"]
    out["red_auto_top_cycles"] = out["red_auto_top_cubes"] + out["red_auto_top_cones"]
    out["blue_auto_top_cycles"] = out["blue_auto_top_cubes"] + out["blue_auto_top_cones"]
    out["red_auto_bot_points"] = 3 * out["red_auto_bot_cycles"]
    out["blue_auto_bot_points"] = 3 * out["blue_auto_bot_cycles"]
    out["red_auto_mid_points"] = 4 * out["red_auto_mid_cycles"]
    out["blue_auto_mid_points"] = 4 * out["blue_auto_mid_cycles"]
    out["red_auto_top_points"] = 6 * out["red_auto_top_cycles"]
    out["blue_auto_top_points"] = 6 * out["blue_auto_top_cycles"]
    
    out["red_teleop_bot_cycles"] = out["red_teleop_bot_cubes"] + out["red_teleop_bot_cones"]
    out["blue_teleop_bot_cycles"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_bot_cones"]
    out["red_teleop_mid_cycles"] = out["red_teleop_mid_cubes"] + out["red_teleop_mid_cones"]
    out["blue_teleop_mid_cycles"] = out["blue_teleop_mid_cubes"] + out["blue_teleop_mid_cones"]
    out["red_teleop_top_cycles"] = out["red_teleop_top_cubes"] + out["red_teleop_top_cones"]
    out["blue_teleop_top_cycles"] = out["blue_teleop_top_cubes"] + out["blue_teleop_top_cones"]
    out["red_teleop_bot_points"] = 2 * out["red_teleop_bot_cycles"]
    out["blue_teleop_bot_points"] = 2 * out["blue_teleop_bot_cycles"]
    out["red_teleop_mid_points"] = 3 * out["red_teleop_mid_cycles"]
    out["blue_teleop_mid_points"] = 3 * out["blue_teleop_mid_cycles"]
    out["red_teleop_top_points"] = 5 * out["red_teleop_top_cycles"]
    out["blue_teleop_top_points"] = 5 * out["blue_teleop_top_cycles"]
    
    out["red_bot_cycles"] = out["red_auto_bot_cycles"] + out["red_teleop_bot_cycles"]
    out["blue_bot_cycles"] = out["blue_auto_bot_cycles"] + out["blue_teleop_bot_cycles"]
    out["red_mid_cycles"] = out["red_auto_mid_cycles"] + out["red_teleop_mid_cycles"]
    out["blue_mid_cycles"] = out["blue_auto_mid_cycles"] + out["blue_teleop_mid_cycles"]
    out["red_top_cycles"] = out["red_auto_top_cycles"] + out["red_teleop_top_cycles"]
    out["blue_top_cycles"] = out["blue_auto_top_cycles"] + out["blue_teleop_top_cycles"]
    out["red_bot_points"] = out["red_auto_bot_points"] + out["red_teleop_bot_points"]
    out["blue_bot_points"] = out["blue_auto_bot_points"] + out["blue_teleop_bot_points"]
    out["red_mid_points"] = out["red_auto_mid_points"] + out["red_teleop_mid_points"]
    out["blue_mid_points"] = out["blue_auto_mid_points"] + out["blue_teleop_mid_points"]
    out["red_top_points"] = out["red_auto_top_points"] + out["red_teleop_top_points"]
    out["blue_top_points"] = out["blue_auto_top_points"] + out["blue_teleop_top_points"]
    
    return out

processed_matches = sorted([process_match(m, w) for m, w in all_matches if m["score_breakdown"] is not None and w is not None], key=lambda x: x["time"])
week_one_matches = [m for m in processed_matches if m["week"] == 1]

In [108]:
for match in processed_matches:
    assert match["red_bot_cycles"] + match["red_mid_cycles"] + match["red_top_cycles"] == match["red_total_cycles"]
    assert match["blue_bot_cycles"] + match["blue_mid_cycles"] + match["blue_top_cycles"] == match["blue_total_cycles"]

In [142]:
keys = [
    "auto_cycles",
    "auto_points",
    "teleop_cycles",
    "teleop_points",
    "total_cycles",
    "links",
    "link_points",
    "total_points",
    "cube_cycles",
    "cube_points",
    "cone_cycles",
    "cone_points",
    "bot_cycles",
    "bot_points",
    "mid_cycles",
    "mid_points",
    "top_cycles",
    "top_points",
]

team_epas = defaultdict(dict)

for key in keys:
    stats = [m["red_" + key] for m in week_one_matches] + [m["blue_" + key] for m in week_one_matches]
    mean, sd = sum(stats) / len(stats), statistics.stdev(stats)

    for team in starting_epas:
        team_epas[team][key] = [mean / 3 + sd * starting_epas[team]]

# Normalize auto/teleop, bot/mid/top, cube/cone to sum to total, keeping original ratio
for team in team_epas:
    for components, total in [
        (("auto_cycles", "teleop_cycles"), "total_cycles"),
        (("auto_points", "teleop_points", "link_points"), "total_points"),
        (("cube_cycles", "cone_cycles"), "total_cycles"),
        (("cube_points", "cone_points", "link_points"), "total_points"),
        (("bot_cycles", "mid_cycles", "top_cycles"), "total_cycles"),
        (("bot_points", "mid_points", "top_points", "link_points"), "total_points"),
    ]:
        comp_sum = sum([max(team_epas[team][key][-1], 0) for key in components if key != "link_points"])
        total_sum = team_epas[team][total][-1] - (team_epas[team]["link_points"][-1] if "link_points" in components else 0)
        for component in components:
            if component != "link_points":
                team_epas[team][component][-1] *= total_sum / max(comp_sum, 1)
        
for key in keys:
    for match in processed_matches:
        # if match["playoff"]:
        #     continue

        red_teams = match["red"]
        blue_teams = match["blue"]

        if max(max(red_teams), max(blue_teams)) > 9500:
            continue

        red_pred = sum(team_epas[x][key][-1] for x in red_teams)
        blue_pred = sum(team_epas[x][key][-1] for x in blue_teams)

        red_actual = match["red_" + key]
        blue_actual = match["blue_" + key]

        weight = 1/3 if match["playoff"] else 1

        red_error = red_actual - red_pred
        for team in red_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            team_epas[team][key].append(team_epas[team][key][-1] + weight * percent * red_error / 3)

        blue_error = blue_actual - blue_pred
        for team in blue_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            team_epas[team][key].append(team_epas[team][key][-1] + weight * percent * blue_error / 3)
        
end_epas = {
    k: {
    "country": locations[k][0],
    "state": locations[k][1],
    "district": locations[k][2],
    } 
    for k in team_epas
}

for k, v in team_epas.items():
    for k2, v2 in v.items():
        end_epas[k][k2] = v2[-1]
    
print(end_epas[254])

{'country': 'USA', 'state': 'CA', 'district': None, 'auto_cycles': 2.0074906276017055, 'auto_points': 10.955184912049134, 'teleop_cycles': 8.740978901372817, 'teleop_points': 31.140434946453464, 'total_cycles': 10.748460147574784, 'links': 3.221777804126537, 'link_points': 16.108889020632688, 'total_points': 58.20450627415554, 'cube_cycles': 4.84971726783189, 'cube_points': 15.926794742284976, 'cone_cycles': 5.907946262561184, 'cone_points': 26.17206404618348, 'bot_cycles': 3.073573500551473, 'bot_points': 6.527991269119564, 'mid_cycles': 2.288485193712691, 'mid_points': 6.984806493656637, 'top_cycles': 5.3856850404710706, 'top_points': 28.73121098632091}


In [143]:
df = pd.DataFrame(end_epas).T

In [144]:
df.sort_values(by=["total_points"], ascending=False).head(10)

,country,state,district,auto_cycles,auto_points,teleop_cycles,teleop_points,total_cycles,links,link_points,...,cube_cycles,cube_points,cone_cycles,cone_points,bot_cycles,bot_points,mid_cycles,mid_points,top_cycles,top_points
2056,Canada,ON,ont,2.443668,12.047851,7.206146,29.593181,9.649813,3.323499,16.617494,...,3.341648,13.537301,6.308166,28.106913,0.63584,1.409228,3.646185,12.034812,5.405174,28.244353
254,USA,CA,None,2.007491,10.955185,8.740979,31.140435,10.74846,3.221778,16.108889,...,4.849717,15.926795,5.907946,26.172064,3.073574,6.527991,2.288485,6.984806,5.385685,28.731211
2910,USA,WA,pnw,2.269926,13.01472,7.417115,27.248263,9.687042,3.310922,16.554611,...,4.357751,15.766517,5.329524,24.499297,2.262693,4.572016,1.82565,5.698749,5.602071,30.002174
1323,USA,CA,None,2.232375,9.231881,8.134682,30.229736,10.367057,3.199264,15.996321,...,5.329069,16.743076,5.037976,22.716564,2.559617,5.085322,3.536354,12.82897,4.262381,21.534717
1678,USA,CA,None,1.610624,8.966435,8.116143,28.900989,9.726772,3.199314,15.99657,...,5.007367,16.002766,4.715499,21.867577,3.059412,6.463431,1.586091,4.583187,5.054503,26.721446
6329,USA,ME,ne,1.816223,10.577521,6.886559,28.181149,8.702781,2.847536,14.237681,...,3.154878,13.15533,5.547747,25.608688,0.819068,1.537938,2.054967,6.466639,5.826761,30.753631
3005,USA,TX,fit,1.386718,7.501485,8.092407,30.566761,9.479125,2.914863,14.574314,...,2.115138,10.068666,7.365475,28.07653,0.402055,1.023784,4.732161,14.190348,4.349563,22.960896
3538,USA,MI,fim,1.292499,7.244374,7.459096,29.827001,8.751596,2.766737,13.833687,...,2.204943,10.330052,6.546665,26.739957,1.398009,2.935265,1.898921,5.742041,5.459979,28.397664
3683,Canada,ON,ont,2.033626,9.548747,6.879669,26.979535,8.913295,2.869186,14.345928,...,2.992477,12.371222,5.920824,24.147377,0.544111,1.271165,4.225166,13.76778,4.1375,21.476955
1577,Israel,None,isr,1.005497,6.010665,8.44755,28.603037,9.453048,3.210577,16.052887,...,3.877916,12.610136,5.575144,22.007976,2.588857,5.181394,2.945035,8.833167,3.920995,20.605729


In [139]:
df.to_json("../../backend/data/epa_breakdown.json", orient="index")

In [151]:
percentile_stats = {}
for key in keys:
    percentile_stats[key] = {
        "p99": df[key].quantile(0.99),
        "p95": df[key].quantile(0.95),
        "p90": df[key].quantile(0.90),
        "p75": df[key].quantile(0.75),
        "p50": df[key].quantile(0.50),
        "p25": df[key].quantile(0.25),
        "mean": df[key].mean(),
        "sd": df[key].std()
    }
    
percentile_df = pd.DataFrame(percentile_stats).T
percentile_df.to_json("../../backend/data/epa_breakdown_percentiles.json", orient="index")